In [13]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.pipeline import make_pipeline

In [14]:
data = pd.read_csv('responses.csv',index_col=0)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('response',axis=1),data.response,random_state=0)

In [16]:
lr = LinearRegression()
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

0.40545271351361967

In [17]:
tree = DecisionTreeRegressor(max_depth=8)
tree.fit(X_train,y_train)
tree.score(X_test,y_test)

0.4213478910669768

In [18]:
rf = RandomForestRegressor(max_depth=12,n_estimators=10)
rf.fit(X_train,y_train)
rf.score(X_test,y_test)

0.4986736924903604

In [19]:
correlations = list(data.corr()['response'][1:])
tree_importances = list(tree.feature_importances_)
rf_importances = list(rf.feature_importances_)
results = pd.DataFrame(data=[correlations,tree_importances,rf_importances],\
             columns=list(data.drop('response',axis=1).columns),\
             index=['correlations','tree_importances','rf_importances'])

results = results.transpose().round(2)
results = results.iloc[results['correlations'].abs().argsort()].iloc[::-1]
results

,correlations,tree_importances,rf_importances
below_poverty_level,-0.44,0.38,0.27
make_over_75k,0.39,0.09,0.09
white,0.36,0.07,0.07
no_high_school,-0.34,0.01,0.03
black,-0.34,0.03,0.04
pop,0.26,0.07,0.06
other_race,-0.19,0.03,0.03
hispanic,-0.19,0.02,0.03
noncitizen,-0.15,0.00,0.02
spanish_poor_english,-0.15,0.00,0.01
